In [6]:
import pandas as pd
import numpy as np
from conct_db import enginex
import calendar, datetime
from datetime import date
from sqlalchemy.types import DATE

In [7]:
periodo = datetime.datetime.now()
mes = periodo.month

In [8]:
df1 = pd.read_excel('y:/19-POWER BI/01-Total Bebidas/TACO GQ/Desafios Agosto.xlsx', skipfooter=3)
df1 = df1[(df1['Distribuidor'] != 'Total') & (df1['ACCOUNT_ID'] != 'Total')]
df1['Periodo'] = pd.to_datetime(date(periodo.year,mes,1))
df1['COD_CLIENTE'] = df1.ACCOUNT_ID.str.extract('(0{2,}\d+)')
df1['COD_CLIENTE'] = df1['COD_CLIENTE'].astype('Int64')
df1.rename({'Código Distribuidor':'COD_DISTRI'}, axis=1, inplace=True)
df1.drop(['Cartera','ACCOUNT_ID', 'Distribuidor'], axis=1, inplace=True)
df1 = df1[['Periodo','COD_DISTRI', 'COD_CLIENTE', 'Visualizaron', 'Aceptaron', '%Aceptaron', 'Completaron', '%Completaron']]

El siguiente codigo extrae los ultimos numeros despues del agregado de ceros intermedio. falta pulirlo  
    
    df1['COD_CLIENTE'] = df1.ACCOUNT_ID.str.extract('(0{2,}\d+)')
    
    df1['COD_CLIENTE'] = df1.COD_CLIENTE.str.extract('([^0{,}]\d+(?:.\d+)?)') 

Para extraer numeros de un texto/numero utilizar la expresion regular 
    
    ('(\d+(?:\.\d+)?)')
    



In [9]:
# df1[pd.isna(df1['Código Distribuidor'])].head

In [10]:
# parameters = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
#                                      "Server=WIN-SERVIDOR-BD\DWGRUPOTOTAL;"
#                                      "DATABASE=DW_Central;"
#                                      "UID=sa;"
#                                      "PWD=DWpln21")

def run():
        # try:
        #         engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(parameters))
        #         print("Conect succesfull")
        # except Exception:
        #         print("Errora in DataBase conection")
        try:
                df1.to_excel('y:/19-POWER BI/01-Total Bebidas/TACO GQ/Desafios Agosto_procesado.xlsx', index=False)
                df1.to_sql('BEES_Desafios', con=enginex, index=False, if_exists='append', dtype={'Periodo': DATE})
                print("Carga exitosa")
        except Exception:
                print("Error al cargar los datos")
text=("""DELETE FROM BEES_Desafios
    WHERE Periodo = '2022-08-01'""")
enginex.execute(text)
run()


Carga exitosa
